# Personalized Learning Demo - Quickstart

A full-stack A2UI sample demonstrating personalized educational content generation.

**Contributed by Google Public Sector's Rapid Innovation Team.**

---

## What This Demo Shows

This demo showcases how A2UI enables AI agents to generate rich, interactive learning materials tailored to individual learners:

- **Flashcards** - Generated from OpenStax textbook content
- **Audio** - Personalized podcasts (via NotebookLM)
- **Video** - Educational explainers
- **Quizzes** - Interactive assessment

### The Personalization Pipeline

At Google Public Sector, we're developing approaches that combine LLMs, knowledge graphs, and learner performance data to produce personalized content across courses—and across a person's academic and professional life.

For this demo, that personalization is represented by context files in `learner_context/` describing a fictional learner (Maria) and her learning needs.

---

## Prerequisites

- Google Cloud project with billing enabled
- `gcloud` CLI installed
- Node.js 18+
- Python 3.11+

## Step 1: Set Your Project ID

Replace with your GCP project ID:

In [15]:
PROJECT_ID = "a2ui-learning"  # <-- CHANGE THIS
LOCATION = "us-central1"

## Step 2: Authenticate & Enable APIs

In [ ]:
# Authenticate with Google Cloud
!gcloud auth login
!gcloud config set project {PROJECT_ID}
!gcloud auth application-default login

In [ ]:
# Enable required APIs
!gcloud services enable aiplatform.googleapis.com
!gcloud services enable cloudbuild.googleapis.com
!gcloud services enable storage.googleapis.com
!gcloud services enable cloudresourcemanager.googleapis.com

In [ ]:
# Create staging bucket for Agent Engine (if it doesn't exist)
!gsutil mb -l {LOCATION} gs://{PROJECT_ID}_cloudbuild 2>/dev/null || echo "Bucket already exists"

## Step 3: Deploy the A2UI Agent

The agent generates personalized learning content and runs on Vertex AI Agent Engine.

In [17]:
# Install Python dependencies (using PyPI explicitly to avoid corporate proxy issues)
!pip install -q --index-url https://pypi.org/simple/ google-adk google-genai vertexai python-dotenv litellm


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Users/samgoodgame/Desktop/student-pls/.venv/bin/python -m pip install --upgrade pip


In [28]:
# Deploy the agent (takes 2-5 minutes)
!cd agent && ../.venv/bin/python deploy.py --project {PROJECT_ID} --location {LOCATION}

Deploying Personalized Learning Agent...
  Project: a2ui-learning
  Location: us-central1
Identified the following requirements: {'cloudpickle': '3.1.2', 'pydantic': '2.12.5', 'google-cloud-aiplatform': '1.130.0'}
The following requirements are missing: {'pydantic', 'google-cloud-aiplatform'}
The following requirements are incompatible: {'cloudpickle==3.1.2 (required: ==3.1.1)'}
The following requirements are appended: {'pydantic==2.12.5'}
The final list of requirements: ['google-adk>=1.15.1', 'google-genai', 'cloudpickle==3.1.1', 'python-dotenv', 'litellm', 'pydantic==2.12.5']
Using bucket a2ui-learning_cloudbuild
Wrote to gs://a2ui-learning_cloudbuild/agent_engine/agent_engine.pkl
Writing to gs://a2ui-learning_cloudbuild/agent_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://a2ui-learning_cloudbuild/agent_engine/dependencies.tar.gz
Creating AgentEngine
Create AgentEngine backing LRO: projects/103904989366/locations/us-central1/reasoningEngines/5377

**Save the Resource ID from the output above!** You'll need it in the next step.

In [31]:
# Get your project NUMBER (different from project ID)
import subprocess
result = subprocess.run(["gcloud", "projects", "describe", PROJECT_ID, "--format=value(projectNumber)"], 
                       capture_output=True, text=True)
PROJECT_NUMBER = result.stdout.strip()
print(f"Project Number: {PROJECT_NUMBER}")

Project Number: 103904989366


## Step 4: Configure Environment

Fill in the Resource ID from the deployment output:

In [32]:
AGENT_RESOURCE_ID = "5377071455685050368"  # <-- PASTE YOUR RESOURCE ID HERE (from Step 3 output)

In [33]:
# Create .env file
env_content = f"""# Generated by Quickstart.ipynb
GOOGLE_CLOUD_PROJECT={PROJECT_ID}
AGENT_ENGINE_PROJECT_NUMBER={PROJECT_NUMBER}
AGENT_ENGINE_RESOURCE_ID={AGENT_RESOURCE_ID}
"""

with open(".env", "w") as f:
    f.write(env_content)

print("Created .env file:")
print(env_content)

Created .env file:
# Generated by Quickstart.ipynb
GOOGLE_CLOUD_PROJECT=a2ui-learning
AGENT_ENGINE_PROJECT_NUMBER=103904989366
AGENT_ENGINE_RESOURCE_ID=5377071455685050368



## Step 5: Install Frontend Dependencies

In [34]:
# Build the A2UI renderer first (using public npm registry)
!cd ../../renderers/lit && npm install --registry https://registry.npmjs.org/ && npm run build

npm warn Unknown project config "always-auth" (//us-npm.pkg.dev/oss-exit-gate-prod/a2ui--npm/:always-auth). This will stop working in the next major version of npm.
npm warn Unknown global config "always-auth" (//us-npm.pkg.dev/artifact-foundry-prod/ah-3p-staging-npm/:always-auth). This will stop working in the next major version of npm.
⠙⠹⠸
up to date, audited 99 packages in 453ms
⠸
⠸12 packages are looking for funding
⠸  run `npm fund` for details
⠸
found 0 vulnerabilities
⠸npm warn Unknown project config "always-auth" (//us-npm.pkg.dev/oss-exit-gate-prod/a2ui--npm/:always-auth). This will stop working in the next major version of npm.
npm warn Unknown global config "always-auth" (//us-npm.pkg.dev/artifact-foundry-prod/ah-3p-staging-npm/:always-auth). This will stop working in the next major version of npm.

> @a2ui/lit@0.8.1 build
> wireit

✅ Ran 0 scripts and skipped 2 in 0s.
⠙

In [35]:
# Install demo dependencies (using public npm registry)
!npm install --registry https://registry.npmjs.org/

npm warn Unknown global config "always-auth" (//us-npm.pkg.dev/artifact-foundry-prod/ah-3p-staging-npm/:always-auth). This will stop working in the next major version of npm.
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴npm warn deprecated node-domexception@1.0.0: Use your platform's native DOMException instead
⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 195 packages, and audited 197 packages in 10s
⠙
⠙30 packages are looking for funding
⠙  run `npm fund` for details
⠙
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠙

## Step 6: Run the Demo

Run this in your terminal (not in the notebook):

```bash
npm run dev
```

Then open http://localhost:5174

### Try These Prompts

| Prompt | What Happens |
|--------|-------------|
| "Help me understand ATP" | Generates flashcards from OpenStax |
| "Quiz me on bond energy" | Interactive quiz cards |
| "Play the podcast" | Audio player (requires Step 7) |
| "Show me a video" | Video player (requires Step 7) |

---

## Step 7 (Optional): Generate Audio & Video with NotebookLM

The demo includes audio and video players, but you need to generate the media files. NotebookLM can create personalized podcasts based on the learner context.

### Prerequisites

- A Google account with access to [NotebookLM](https://notebooklm.google.com/)
- The `learner_context/` files from this demo

---

### Part A: Generate a Personalized Podcast

**1. Create a NotebookLM Notebook**

Go to [notebooklm.google.com](https://notebooklm.google.com/) and create a new notebook.

**2. Upload Learner Context Files**

Upload all files from the `learner_context/` directory:
- `01_maria_learner_profile.txt` - Maria's background and learning preferences  
- `02_chemistry_bond_energy.txt` - Bond energy concepts
- `03_chemistry_thermodynamics.txt` - Thermodynamics content
- `04_biology_atp_cellular_respiration.txt` - ATP and cellular respiration
- `05_misconception_resolution.txt` - Common misconceptions to address
- `06_mcat_practice_concepts.txt` - MCAT-focused content

These files give NotebookLM the context to generate personalized content.

**3. Generate the Audio Overview**

- Click **Notebook guide** in the right sidebar
- Click **Audio Overview** → **Generate**
- Wait for generation to complete (typically 2-5 minutes)
- NotebookLM will create a podcast-style discussion about the uploaded content

**4. Customize the Audio (Optional)**

Before generating, you can click **Customize** to provide specific instructions:

```
Create a podcast for Maria, a pre-med student preparing for the MCAT. 
Use gym and fitness analogies since she loves working out.
Focus on explaining why "energy stored in bonds" is a misconception.
Make it conversational and engaging, about 5-7 minutes long.
```

**5. Download and Install the Podcast**

- Once generated, click the **⋮** menu on the audio player
- Select **Download**
- Save the file as `podcast.m4a`
- Copy to the demo's assets directory:

In [ ]:
# Copy your downloaded podcast to the assets directory
# Replace ~/Downloads/podcast.m4a with your actual download path
!cp ~/Downloads/podcast.m4a public/assets/podcast.m4a

# Verify the file was copied
!ls -la public/assets/

---

### Part B: Create a Video (Two Options)

#### Option 1: NotebookLM Briefing Document + Screen Recording

**1. Generate a Briefing Document**

In NotebookLM with your learner context loaded:
- Click **Notebook guide** → **Briefing doc**
- This creates a structured summary you can use as a video script

**2. Create the Video**

Use the briefing document to create a video:
- **Screen recording** - Record yourself walking through the concepts using slides or a whiteboard app
- **AI video tools** - Use tools like Synthesia, HeyGen, or similar to generate a video from the script
- **Slide presentation** - Create slides and record with voiceover using QuickTime, Loom, or similar

**3. Export and Install**

Export your video as MP4 and copy to the assets directory:

In [ ]:
# Copy your video to the assets directory
# Replace ~/Downloads/demo.mp4 with your actual file path
!cp ~/Downloads/demo.mp4 public/assets/demo.mp4

# Verify both media files are in place
!ls -la public/assets/

#### Option 2: Use Placeholder/Stock Content

For demo purposes, you can use any MP4 video file. Rename it to `demo.mp4` and place it in `public/assets/`.

---

### Verify Media Files

After copying your files, verify they're accessible:

In [ ]:
import os

print("Media files status:")
print("-" * 40)

podcast_path = "public/assets/podcast.m4a"
video_path = "public/assets/demo.mp4"

if os.path.exists(podcast_path):
    size_mb = os.path.getsize(podcast_path) / (1024 * 1024)
    print(f"✅ Podcast: {podcast_path} ({size_mb:.1f} MB)")
else:
    print(f"❌ Podcast: {podcast_path} NOT FOUND")
    
if os.path.exists(video_path):
    size_mb = os.path.getsize(video_path) / (1024 * 1024)
    print(f"✅ Video: {video_path} ({size_mb:.1f} MB)")
else:
    print(f"❌ Video: {video_path} NOT FOUND")

print("-" * 40)
print("\nRun 'npm run dev' and try:")
print('  • "Play the podcast" - to hear the audio')
print('  • "Show me a video" - to watch the video')

---

## Content Attribution

### OpenStax

Educational content is sourced from [OpenStax](https://openstax.org/), licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/).

Specifically: [Biology for AP® Courses](https://openstax.org/details/books/biology-ap-courses) - OpenStax, Rice University

---

## Security Notice

> **Warning:** When building production applications, treat any agent outside your control as potentially untrusted. This demo connects to Agent Engine within your own GCP project. Always review agent code before deploying.